In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import json, os, collections, ast

In [2]:
playtest_files = ["2018-11-14_Playtest/anonymized_playtest5_data.csv", "2019-01-07_Playtest/anonymized_playtest6_data.csv", "2019-01-31_Playtest/anonymized_playtest7_data.csv"]

playtests = []

for playtest in playtest_files:
    playtests.append(pd.read_csv(playtest, sep=";", index_col=0))
    
# Set the playtest index we want to focus on
playtest = 1


In [3]:
def get_shapes(puzzle):
    shapes = []
    if puzzle is None:
        return shapes
    for shape_data in puzzle.get("shapeData"):
        shapes.append(shape_data.get('shapeType'))
    return shapes    

In [4]:
puzzles = collections.OrderedDict()

with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/config.json") as f:
    asset_config = json.load(f)

for puzzle_sets in asset_config.get("puzzleSets"):
    for puzzle_file in puzzle_sets.get("puzzles"):
        with open(f"{os.path.dirname(playtest_files[playtest])}/StreamingAssets/{puzzle_file}.json") as f:
            puzzle_details = json.load(f)
            puzzles[puzzle_details.get("puzzleName")] = puzzle_details

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
#all_playtest_data = pd.concat([playtest5_data, playtest6_data, playtest7_data])
all_playtest_data = pd.concat([playtests[playtest]])

#all_playtest_data = all_playtest_data.join(all_playtest_data['data'].map(json.loads).apply(pd.Series).add_prefix('data.')).drop(["data"], axis=1)
all_playtest_data = all_playtest_data.join(all_playtest_data['data'].map(json.loads).apply(pd.Series)).drop(["data"], axis=1)

In [6]:
# puzzleDict[user~puzzle_id] = {}
puzzleDict = {}
frames = []

# For each session
for user in all_playtest_data['session_id'].unique():
    # Get the sessions
    user_events = all_playtest_data[all_playtest_data['session_id'] == user]
    activePuzzle = None
    for enum, event in user_events.iterrows():
        #print(('{} - {}').format(event['time'], event['type']))
        # Keep track of all frames we've looked at
        if(event['type'] == 'ws-start_level'):
            activePuzzle = event['task_id']
            frames.clear()

            
        # If they are not playing a puzzle we do not do anything and continue
        if(activePuzzle is None):
            continue

    # Analyze when puzzle is finished or user left
        # Measure time, attempts, completion and actions
        if(event['type'] in ['ws-exit_to_menu', 'ws-disconnect']):
            # This was incomplete so don't reset the active puzzle
            activePuzzle = None
            frames.clear()

        elif (event['type'] in ['ws-puzzle_complete']):
            # This was a correct solution! We need to mark all of these frames as correct
            for frame in frames:
                all_playtest_data.at[frame, 'completed'] = True
            
        else:
            # Set the active puzzle to be this one we're looking at
            all_playtest_data.at[enum, 'task_id'] = activePuzzle
            frames.append(enum)


In [7]:
# This was part of a failed effort to get the only the data for completed attempts 

all_playtest_data['time'] = pd.to_datetime(all_playtest_data['time'])
all_playtest_data.query('completed == True')


,time,type,session_id,game_id,group,version_num,env_configs,user,timeStamp,screenPos,actionIndex,task_id,set_id,fullscreen,resolution,conditions,mode,prevMode,prevXfmMode,xfmMode,prevShape,shape,input,shapeType,objSerialization,spawnPosition,color,changeMode,newSelection,prevSelection,selectedObject,rotationOffset,correct,selectedObjects,prevPositions,targetOffset,validMove,deletedShapes,targetCam,prevCam,cameraAngle,prevScale,newScale,targetedActionIndex,isOpen,newPalette,prevPalette,shapeIndex,newColor,prevColor,completed
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
24165,2019-01-07 09:43:58.329554,ws-start_level,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,NaN,NaN,NaN,Rotate a Pyramid,Tutorial,False,"{'x': 960, 'y': 540}","{""shapeLimits"":[-1,0,-1,0,0,0,0],""allowScale"":false,""allowRotate"":true,""gridDim"":3}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24166,2019-01-07 09:43:58.491723,ws-puzzle_started,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,0.000000,"{'x': 0.840624988079071, 'y': 0.4870370328426361}",1.0,Rotate a Pyramid,Tutorial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24167,2019-01-07 10:50:00.954384,ws-mode_change,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,3962.644287,"{'x': 0.41979166865348816, 'y': 0.03703703731298447}",2.0,Rotate a Pyramid,NaN,NaN,NaN,NaN,3.0,1.0,1.0,1.0,6.0,2.0,LeftMouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24168,2019-01-07 10:50:01.583632,ws-create_shape,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,3963.262207,"{'x': 0.5197916626930237, 'y': 0.35185185074806213}",3.0,Rotate a Pyramid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LeftMouse,2.0,1.0,"{'x': 2, 'y': 0, 'z': 2}",1.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24169,2019-01-07 10:50:03.197463,ws-mode_change,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,3964.879395,"{'x': 0.11145833134651184, 'y': 0.09259258955717087}",4.0,Rotate a Pyramid,NaN,NaN,NaN,NaN,1.0,1.0,1.0,2.0,2.0,2.0,LeftMouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24170,2019-01-07 10:50:03.842650,ws-rotate_shape,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,3965.529785,"{'x': 0.44062501192092896, 'y': 0.23333333432674408}",5.0,Rotate a Pyramid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LeftMouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"{'x': 0, 'y': 1, 'z': 0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24171,2019-01-07 10:50:08.138745,ws-check_solution,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,3969.824219,"{'x': 0.5510416626930237, 'y': -0.003703703638166189}",6.0,Rotate a Pyramid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[True, True]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24214,2019-01-07 13:38:43.104110,ws-start_level,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,NaN,NaN,NaN,Match Silhouettes,Tutorial,True,"{'x': 1920, 'y': 1080}","{""shapeLimits"":[-1,-1,-1,0,0,0,0],""allowScale"":true,""allowRotate"":false,""gridDim"":3}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
24215,2019-01-07 13:38:43.496587,ws-puzzle_started,ohait6vg1n9hr94sw7pcfe91diibz0xv-Playtest6,NaN,,NaN,NaN,guest,0.000000,"{'x': 0.3630208373069763, 'y': 0.5046296119689941}",1.0,Match Silhouettes,Tutorial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [8]:
all_attempts = all_playtest_data.task_id.value_counts()


In [9]:
complete_puzzles = all_playtest_data.loc[all_playtest_data['type'] == 'ws-puzzle_complete']
successful_attempts = complete_puzzles.task_id.value_counts()

In [10]:
#created_shapes = all_playtest_data.loc[all_playtest_data['type'] == 'ws-create_shape']
created_shapes = all_playtest_data.query('type == "ws-create_shape"')
success_created_shapes =  created_shapes.query('completed == True')

#failure_created_shapes = all_playtest_data.loc[all_playtest_data['type'] == 'ws-create_shape' & all_playtest_data['completed']]

shapes_used = collections.defaultdict(list)
for shapes, value in created_shapes.iterrows():
    shapes_used[value.get('task_id')].append(int(value.get('shapeType')))
    
shapes_used_success = collections.defaultdict(list)
for shapes, value in success_created_shapes.iterrows():
    shapes_used_success[value.get('task_id')].append(int(value.get('shapeType')))

In [11]:
for attempt, value in all_attempts.items():
    success_value = successful_attempts.get(attempt, 0)
    shapes = get_shapes(puzzles.get(attempt))
    print (f"Puzzle {attempt: <25} Attempts {value: < 5} / Success {success_value: < 5} : {round(((success_value/value) * 100), 1): < 5}% Shapes: {shapes}")
    times_used = collections.Counter(shapes_used[attempt])
    print (f"Shapes Used Total: {times_used}")
    times_used = collections.Counter(shapes_used_success[attempt])
    print (f"Shapes Used Success: {times_used}")


Puzzle Design a Puzzle           Attempts  1257 / Success  0    :  0.0 % Shapes: []
Shapes Used Total: Counter({1: 33, 4: 23, 5: 18, 2: 11, 3: 6, 6: 5})
Shapes Used Success: Counter({1: 3, 4: 3, 2: 3, 5: 1, 3: 1})
Puzzle Pyramids are Strange      Attempts  1050 / Success  6    :  0.6 % Shapes: [2, 2, 2, 2]
Shapes Used Total: Counter({2: 48})
Shapes Used Success: Counter({2: 31})
Puzzle Scaling Round Objects     Attempts  767  / Success  7    :  0.9 % Shapes: [4, 2]
Shapes Used Total: Counter({4: 13, 5: 13, 6: 1, 1: 1})
Shapes Used Success: Counter({4: 10, 5: 9, 6: 1})
Puzzle Cubes Obscure Spheres     Attempts  724  / Success  6    :  0.8 % Shapes: [6, 4, 4, 1]
Shapes Used Total: Counter({1: 33, 4: 27, 6: 11})
Shapes Used Success: Counter({1: 14, 4: 13, 6: 7})
Puzzle Not Bird                  Attempts  647  / Success  3    :  0.5 % Shapes: [6, 4, 2]
Shapes Used Total: Counter({1: 10, 4: 9, 6: 6, 5: 6, 2: 5, 3: 2})
Shapes Used Success: Counter({4: 5, 5: 4, 2: 3, 6: 2})
Puzzle Bird Fez   

In [12]:

all_playtest_data.type.value_counts()

ws-mode_change                1520
ws-rotate_shape               1202
ws-rotate_view                1088
ws-scale_shape                1004
ws-move_shape                 958 
ws-select_shape               927 
ws-deselect_shape             832 
ws-create_shape               649 
ws-puzzle_started             267 
ws-start_level                245 
ws-check_solution             243 
ws-click_nothing              237 
ws-exit_to_menu               200 
ws-snapshot                   194 
ws-delete_shape               158 
ws-puzzle_complete            143 
ws-undo_action                56  
ws-toggle_snapshot_display    50  
ws-login_user                 22  
ws-start_game                 22  
ws-restart_puzzle             22  
ws-select_shape_add           20  
ws-paint                      14  
ws-toggle_paint_display       12  
ws-palette_change             12  
ws-redo_action                3   
start_game                    1   
Name: type, dtype: int64